<a href="https://www.kaggle.com/code/abriambuggs/rice-classification-99-6-accuracy?scriptVersionId=102981563" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importing some libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import random
import matplotlib.image as mpimg
import shutil

In [ ]:
# Getting the data
!wget https://www.muratkoklu.com/datasets/vtdhnd09.php

In [ ]:
# Extracting the zipped data
import zipfile
zip_ref = zipfile.ZipFile('./vtdhnd09.php')
zip_ref.extractall()
zip_ref.close()

In [ ]:
# Getting the class name
import pathlib

data_dir = pathlib.Path('./Rice_Image_Dataset')
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

In [ ]:
#creating a function to veiw sample images
def view_random_image(target_dir, target_class = None, samples_num=int):
  if target_class == None:
    target_class = class_names[random.randint(0,(len(class_names)-2))]
  else:
    target_class = target_class
  target_folder = str(target_dir) + "/" + str(target_class)
  random_image = random.sample(os.listdir(target_folder), samples_num)
  

  for x in range (0,samples_num):
    img = mpimg.imread(target_folder+"/"+random_image[x])
    plt.figure(figsize=(10,7))
    ax = plt.subplot(1,samples_num, x+1)
    plt.imshow(img)

    ax.title.set_text((target_class,"image shape is: " ,img.shape))

In [ ]:
view_random_image(data_dir,samples_num=12)

In [ ]:
# Veiwing samples of different types of rice to see the variance
def look_at_rice(target_dir):
  for x in class_names[:-1]:
    target_folder = (target_dir+"/"+x)
    random_image = random.sample(os.listdir(target_folder), 1)
    random_image = str(random_image[0])
    plt.figure(figsize = (10,7))
    img = mpimg.imread(target_folder+"/"+random_image)
    #ax = plt.subplot(1,len(class_names[:-1]),x+1)
    plt.imshow(img)
    plt.xlabel=False
    plt.ylabel = False
    plt.title(x)

In [ ]:
look_at_rice("./Rice_Image_Dataset")

In [ ]:
# Making the train and test directories
os.mkdir('./Rice_Image_Dataset/train')
os.mkdir('./Rice_Image_Dataset/test')

In [ ]:
# Making the folder for the different classes of rice
for x in class_names[:-1]:
  os.makedirs('./Rice_Image_Dataset/train/'+x)
  os.mkdir('./Rice_Image_Dataset/test/'+x)


In [ ]:
# Making the test and training sets
def train_test(target_dir, percentage=float):
  for x in class_names[:-1]:
    pathss = target_dir + x
    img_count = round(len(os.listdir(pathss))*percentage)
    old_file = []
    old_file2 = [] 
    new_file = []
    new_file2 = []
    for stuff in os.listdir(pathss):
      old_file.append(stuff)
      old_file2.append(stuff)
    train_set_file= random.sample(os.listdir(pathss), img_count)
    #print(train_set_file)
    #return(train_set_file)
    
    for file in train_set_file:
      paths_1 = pathss + "/" + file
      paths_2 = target_dir + "train/" + x + "/" + file
      #print(paths_1,paths_2)
      shutil.copyfile(paths_1, paths_2)
    for thing in old_file:
      if thing in train_set_file:
        old_file2.remove(thing)
        new_file.append(thing)
      else:
        new_file2.append(thing)
    #return(old_file, new_file, train_set_file, new_file2, old_file2)
    for thing2 in old_file2:
      paths_3 = pathss + "/" + thing2
      paths_4 = target_dir + "test/" + x +"/" + thing2
      shutil.copyfile(paths_3, paths_4)

In [ ]:
work_dir = './Rice_Image_Dataset/'
train_test(work_dir, .8)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255.)
test_datagen = ImageDataGenerator(rescale=1./255.)

train_dir = "./Rice_Image_Dataset/train"
test_dir = "./Rice_Image_Dataset/test"

print("Training images:")
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size = (224,244),
                                               batch_size = 32,
                                               class_mode = "categorical")
print("Testing images:")

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size = (224,224))


In [ ]:
# Using the efficientnet/b0/feature-vector model
efficientnet_url = 'https://tfhub.dev/google/efficientnet/b0/feature-vector/1'

In [ ]:
from tensorflow.keras import layers
import tensorflow_hub as hub

In [ ]:
# Create_model function to create a model from a url
def create_model(model_url, num_class = 5):
  feature_extractor_layer = hub.KerasLayer(handle = model_url,
                                           #trainable = True,
                                           name = "feature_extraction_layer",
                                           input_shape = (224,224,3))
  model = tf.keras.Sequential([
                                feature_extractor_layer,
                                layers.Dense(num_class, activation = "softmax", name="output_layer")
  ])
  return model

In [ ]:
efficient_model = create_model(efficientnet_url, 5)

In [ ]:
efficient_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                        optimizer = tf.keras.optimizers.Adam(),
                        metrics = ["accuracy"])

In [ ]:
history_efficient = efficient_model.fit(train_data,
                                  batch_size =32,
                                  epochs = 8,
                                  steps_per_epoch = len(train_data),
                                  validation_data = test_data,
                                  validation_steps = len(test_data))

In [ ]:
# Save the model
tf.keras.models.save_model(efficient_model,"rice_model")


In [ ]:
import pandas as pd
pd.DataFrame(history_efficient.history).plot()

In [ ]:
# Create a function to import and resize an image to be used for the model
def load_and_prep_image(filename, img_shape=224):
  """
  Reads an image from filename, turns it into a tensor and reshapes it
  to (img_shape, img_shape, colour_channels).
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode the read file into a tensor
  img = tf.image.decode_image(img)
  # REsize the image
  img = tf.image.resize(img, size = [img_shape,img_shape])
  # Rescale the image (get all values between 0 and 1)
  img = img/255.
  return tf.expand_dims(img, axis = 0)

In [ ]:
# Reconfig pred_and_plot function to work with multi-class images
def pred_and_plot_multi(model, filename, class_names=class_names):
  """
  Imports an image located at filename, makes a predction with the model, and plots
  the image with the predicted class as a title.
  """

  # Imports the target images and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(tf.squeeze(tf.expand_dims(img, axis=0)),axis=0))

  # Gets the highest prediction value
  pred = tf.squeeze((np.amax(pred)))

  #Get the predicted class
  pred_class = class_names[int(tf.round(pred))]

  # Plot the image and predicted class
  plt.imshow(tf.squeeze(img))
  plt.title(("Prediction", pred_class, pred))
  plt.axis(False);

In [ ]:
# Test a random image from the internet
!wget "https://i0.wp.com/saucydressings.com/wp-content/uploads/2020/06/basmati-rice.jpg?fit=1024%2C676&ssl=1"

In [ ]:
rice_2 = "./basmati-rice.jpg?fit=1024,676&ssl=1"

In [ ]:
load_and_prep_image(rice_2)

In [ ]:
pred_and_plot_multi(efficient_model,rice_2)